In [93]:
import ee
import pandas as pd
import geopandas as gpd
import json
import pickleshare 
from parameters.config_output_naming import geo_id_column
from modules.gee_initialize import initialize_ee 
import modules.agstack_to_gee as agstack_to_gee
from parameters.config_asr_url_info import asset_registry_base,user_registry_base
from parameters.config_asr_credentials import email,password

initialize_ee()

SyntaxError: invalid syntax (3830311545.py, line 12)

#### List of Geo_ids

In [94]:
GHA_ids = ['88bec54ad04804f5b1fafbc131266640a129be2840fa6797cda358d7e831b907', 
        'ef2f7c46fbe4fc892fdb81f9a31c9c507b9f1e4548504247dcbbab28cf8e436c',
        '97408ef7bdac487e4a42e4abf20492b786310889fd4b0478603e2d0004c40bfb']

IDN_ids = ['c288d6c94efa9011c0e3452af9f7fa0941661377030e10d29c68764617f9816d', 
       '1a41a309ae2387f36a604c9a6c81887e64357a7f61d228758e23ef766286fcd7',
       '1a4472dc40700ef33f931863f58d444f243d64418616678fcf85c57e1f4bbf45',
       '8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53859993522a6f05727a']

CIV_ids = ['0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f665ea83ac9aad3512918',
           'b84f55de2b7f3c77d1cbeb8b026a1b29be42d8b08d92058c9143e0556456820f',
           'b7c15efb6e3c63fcfe649a2d994973a6f5caa844f720f0edb7cf24f6a6c3c1b3',
            'fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b84f5b9445855eef02f']

all_geo_ids= GHA_ids + IDN_ids + CIV_ids

In [75]:
#fetch and convert into feature collection
session = agstack_to_gee.start_agstack_session(email,password,user_registry_base)

roi = agstack_to_gee.geo_id_or_ids_to_feature_collection(
    all_geo_ids=all_geo_ids, 
    geo_id_column=geo_id_column, 
    session=session, 
    asset_registry_base=asset_registry_base,
    required_area=4,area_unit="ha")
 

#### from GEE asset (feature collection) 

In [91]:
example_plot_feature_col_asset = "projects/fdap-remi/assets/selected_segments_peru_vector"
roi = ee.FeatureCollection(example_plot_feature_col_asset)


#### Feature collection

In [88]:
geometry2 = ee.Geometry.Polygon(
        [[[31.115740424411104, -23.849275490814446],
          [31.115740424411104, -24.330688549133132],
          [33.02736151816111, -24.330688549133132],
          [33.02736151816111, -23.849275490814446]]], None, False);

geometry4 = ee.Geometry.Polygon(
        [[[102.18777620396249, 3.079459767638651],
          [102.18831264576546, 3.0785384196951506],
          [102.1895786484205, 3.0792026473624907],
          [102.18899929127328, 3.08005971470826]]],None, False);

# roi=geometry4

# stats= get_stats(roi)#,geo_id_column)

feat_2 = ee.Feature(geometry2, {geo_id_column: "fsgghdgfdgdsdgs"})
feat_4 = ee.Feature(geometry4, {geo_id_column: "hkjkjfhbadfhgdh"})

roi = ee.FeatureCollection([feat_2,feat_4])


#### Shapefile

In [85]:
shp = "test_ceo_all.shp.zip"

# Load your geospatial dataset
gdf = gpd.read_file(shp)

geo_json = gdf.to_json()

roi = ee.FeatureCollection(json.loads(geo_json))

roi = roi.select([geo_id_column])


#### Calculate stats

In [89]:
def stats_as_df (roi): 
    %store roi
    %run alt_whisp_reformat.ipynb   
    return df_out

In [92]:
df_out = stats_as_df(roi)
df_out

Stored 'roi' (FeatureCollection)
stats time: 2.033179521560669 seconds
table tidying time: 22.662867546081543 seconds


,Geo id,Area ha,Country,EUFO 2020,GLAD Primary,TMF undist,JAXA FNF 2020,GFC TC 2020,GLAD LULC 2020,ESA TC 2020,TMF disturbed,RADD alerts,TMF plant,Oil palm Descals,Oil palm FDaP,Cocoa ETH,WDPA
PLOTID,,,,,,,,,,,,,,,,,
1,NaN,3.8,PER,74,0,0,36,2,0,48,0,-,0,0,0,0,-
2,NaN,2.0,PER,47,0,0,59,10,0,58,0,-,0,0,0,0,-
3,NaN,0.1,PER,0,0,0,80,0,0,0,0,-,0,0,0,0,-
4,NaN,0.2,PER,2,0,0,0,9,0,40,0,-,0,0,0,0,-
5,NaN,2.1,PER,1,0,0,5,1,0,6,0,-,0,0,0,0,-
6,NaN,1.5,PER,22,0,0,52,0,0,43,0,-,0,0,0,0,-
7,NaN,1.0,PER,17,0,0,37,0,0,16,0,-,0,0,0,0,-
8,NaN,2.5,PER,0,0,0,67,0,0,6,0,-,0,0,0,0,-
9,NaN,2.5,PER,4,0,0,57,0,0,4,0,-,0,0,0,0,-


#### Export CSV

In [80]:
#export output csv
out_file_wide = "alt_whisp_output_table.csv"

df_out.to_csv(path_or_buf=out_file_wide,header=True)

print ("output csv: ", out_file_wide)

output csv:  alt_whisp_output_table.csv
